In [145]:
import dask.dataframe as dd
import  numpy as np

In [146]:
df = dd.read_csv("DATOS/air_traffic_data.csv")
df
#Información de cada columna
print(df.head())

   Activity Period Operating Airline Operating Airline IATA Code  \
0           200507      ATA Airlines                          TZ   
1           200507      ATA Airlines                          TZ   
2           200507      ATA Airlines                          TZ   
3           200507       Air Canada                           AC   
4           200507       Air Canada                           AC   

  Published Airline Published Airline IATA Code    GEO Summary GEO Region  \
0      ATA Airlines                          TZ       Domestic         US   
1      ATA Airlines                          TZ       Domestic         US   
2      ATA Airlines                          TZ       Domestic         US   
3       Air Canada                           AC  International     Canada   
4       Air Canada                           AC  International     Canada   

  Activity Type Code Price Category Code    Terminal Boarding Area  \
0           Deplaned            Low Fare  Terminal 1      

In [147]:
#Quitamos las columnas innecesarias para el estudio de datos

columnas_innecesarias = ["Adjusted Activity Type Code", "Operating Airline IATA Code", "Published Airline IATA Code", "Activity Type Code"]

# Elimina las columnas innecesarias
df = df.drop(columnas_innecesarias, axis=1)
print(df.head(10))


   Activity Period Operating Airline Published Airline    GEO Summary  \
0           200507      ATA Airlines      ATA Airlines       Domestic   
1           200507      ATA Airlines      ATA Airlines       Domestic   
2           200507      ATA Airlines      ATA Airlines       Domestic   
3           200507       Air Canada        Air Canada   International   
4           200507       Air Canada        Air Canada   International   
5           200507         Air China         Air China  International   
6           200507         Air China         Air China  International   
7           200507        Air France        Air France  International   
8           200507        Air France        Air France  International   
9           200507   Air New Zealand   Air New Zealand  International   

            GEO Region Price Category Code       Terminal Boarding Area  \
0                   US            Low Fare     Terminal 1             B   
1                   US            Low Fare    

In [148]:
#Compruebo si la columna Passenger Count y Adjusted Passenger Count tienen los mismos valores, y no, en 920 ocasiones no tienen los mismos valores las dos columnas

comparacion = df["Adjusted Passenger Count"].eq(df["Passenger Count"])

# sumamos la cantidad de valores false
cantidad_false = (~comparacion).sum().compute()
print(f"En {cantidad_false} casos hay diferencias entre las 2 columnas")



En 920 casos hay diferencias entre las 2 columnas


In [149]:
#920 casos de 15000 no es muy significativo, un 6,13% de imprecision en la columna "Passenger count" que puedo permitir cometer
df = df.drop("Adjusted Passenger Count", axis=1)


In [150]:

comparacion = df["Operating Airline"].eq(df["Published Airline"])

# sumamos la cantidad de valores false
cantidad_false = (~comparacion).sum().compute()
print(f"En {cantidad_false} casos hay diferencias entre las 2 columnas")


En 1602 casos hay diferencias entre las 2 columnas


In [151]:
#1602 casos de 15000 no es muy significativo, un 10.68%. de imprecision en la columna "Published Airline" que puedo permitir cometer
df = df.drop("Published Airline", axis=1)

In [152]:
import dask.dataframe as dd

#calculo los valores unicos de la columna
valores_unicos = df["Operating Airline"].unique().compute()

#asigno un valor unico a cada object diferente
valor_entero = np.arange(len(valores_unicos))
valor_dict = dict(zip(valores_unicos, valor_entero))

#modifico la columna antigua con los nuevos valores
df["Operating Airline"] = df["Operating Airline"].map(lambda x: valor_dict.get(x, np.nan), meta=("Operating Airline", 'int64'))
print(valor_dict)



{'ATA Airlines': 0, 'Air Canada ': 1, 'Air China': 2, 'Air France': 3, 'Air New Zealand': 4, 'AirTran Airways': 5, 'Alaska Airlines': 6, 'All Nippon Airways': 7, 'American Airlines': 8, 'American Eagle Airlines': 9, 'Asiana Airlines': 10, 'Atlantic Southeast Airlines': 11, 'BelAir Airlines': 12, 'British Airways': 13, 'Cathay Pacific': 14, 'China Airlines': 15, 'Delta Air Lines': 16, 'EVA Airways': 17, 'Frontier Airlines': 18, 'Hawaiian Airlines': 19, 'Horizon Air ': 20, 'Icelandair': 21, 'Independence Air': 22, 'Japan Airlines': 23, 'KLM Royal Dutch Airlines': 24, 'Korean Air Lines': 25, 'Lufthansa German Airlines': 26, 'Mesa Airlines': 27, 'Mexicana Airlines': 28, 'Midwest Airlines': 29, 'Northwest Airlines': 30, 'Philippine Airlines': 31, 'Singapore Airlines': 32, 'SkyWest Airlines': 33, 'Sun Country Airlines': 34, 'TACA': 35, 'US Airways': 36, 'United Airlines': 37, 'United Airlines - Pre 07/01/2013': 38, 'Virgin Atlantic': 39, 'WestJet Airlines': 40, 'Boeing Company': 41, 'Miami A

In [153]:

#imprimo los valores unicos de la columna para saber que tengo que reemplazar por valores int
valores_unicos = df["GEO Summary"].unique().compute()
#genera un array NumPy con valores enteros en un rango que va desde 0 hasta la longitud de la lista valores_a_reemplazar. Estos valores enteros serán los nuevos valores a asignar
valores_a_reemplazar = ["Domestic", "International"]
valores_nuevos = np.arange(len(valores_a_reemplazar), dtype=np.int64)
#crea un diccionario que mapea cada valor a reemplazar con su respectivo valor nuevo, la funcion zip() combina los elementos de las dos listas en pares clave-valor, y la función dict() crea un diccionario a partir de esos pares.
mapping_dict_Geo_Summary = dict(zip(valores_a_reemplazar, valores_nuevos))

df["GEO Summary"] = df["GEO Summary"].map(mapping_dict_Geo_Summary)

df["GEO Summary"] = df["GEO Summary"].astype(np.int64)

resultado = df.compute()
print(resultado)


       Activity Period  Operating Airline  GEO Summary GEO Region  \
0               200507                  0            0         US   
1               200507                  0            0         US   
2               200507                  0            0         US   
3               200507                  1            1     Canada   
4               200507                  1            1     Canada   
...                ...                ...          ...        ...   
15002           201603                 53            0         US   
15003           201603                 53            1     Mexico   
15004           201603                 53            1     Mexico   
15005           201603                 39            1     Europe   
15006           201603                 39            1     Europe   

      Price Category Code       Terminal Boarding Area  Passenger Count  Year  \
0                Low Fare     Terminal 1             B            27271  2005   
1        

In [154]:
#hago lo mismo que con la columna "GEO Summary"
valores_unicos = df["GEO Region"].unique().compute()

valores_a_reemplazar = ["US", "Canada", "Asia", "Europe", "Australia / Oceania", "Mexico", "Central America", "Middle East", "South America"]
valores_nuevos = np.arange(len(valores_a_reemplazar), dtype=np.int64)

mapping_dict_Geo_Region = dict(zip(valores_a_reemplazar, valores_nuevos))

df["GEO Region"] = df["GEO Region"].map(mapping_dict_Geo_Region)

df["GEO Region"] = df["GEO Region"].astype(np.int64)

resultado = df.compute()
print(resultado)
print(mapping_dict)



       Activity Period  Operating Airline  GEO Summary  GEO Region  \
0               200507                  0            0           0   
1               200507                  0            0           0   
2               200507                  0            0           0   
3               200507                  1            1           1   
4               200507                  1            1           1   
...                ...                ...          ...         ...   
15002           201603                 53            0           0   
15003           201603                 53            1           5   
15004           201603                 53            1           5   
15005           201603                 39            1           3   
15006           201603                 39            1           3   

      Price Category Code       Terminal Boarding Area  Passenger Count  Year  \
0                Low Fare     Terminal 1             B            27271  2005 

In [155]:
#hago lo mismo que con la columna "GEO Summary"
valores_unicos = df["Price Category Code"].unique().compute()

valores_a_reemplazar = ["Low Fare", "Other"]
valores_nuevos = np.arange(len(valores_a_reemplazar), dtype=np.int64)

mapping_dict_Price_Category_Code = dict(zip(valores_a_reemplazar, valores_nuevos))

df["Price Category Code"] = df["Price Category Code"].map(mapping_dict_Price_Category_Code)

df["Price Category Code"] = df["Price Category Code"].astype(np.int64)

resultado = df.compute()
print(resultado)

       Activity Period  Operating Airline  GEO Summary  GEO Region  \
0               200507                  0            0           0   
1               200507                  0            0           0   
2               200507                  0            0           0   
3               200507                  1            1           1   
4               200507                  1            1           1   
...                ...                ...          ...         ...   
15002           201603                 53            0           0   
15003           201603                 53            1           5   
15004           201603                 53            1           5   
15005           201603                 39            1           3   
15006           201603                 39            1           3   

       Price Category Code       Terminal Boarding Area  Passenger Count  \
0                        0     Terminal 1             B            27271   
1      

In [156]:
#hago lo mismo que con la columna "GEO Summary"
valores_unicos = df["Terminal"].unique().compute()

valores_a_reemplazar = ["Terminal 1", "Terminal 2", "Terminal 3", "International", "Other"]
valores_nuevos = np.arange(len(valores_a_reemplazar), dtype=np.int64)

mapping_dict_Terminal = dict(zip(valores_a_reemplazar, valores_nuevos))

df["Terminal"] = df["Terminal"].map(mapping_dict_Terminal)

df["Terminal"] = df["Terminal"].astype(np.int64)

resultado = df.compute()
print(resultado)


       Activity Period  Operating Airline  GEO Summary  GEO Region  \
0               200507                  0            0           0   
1               200507                  0            0           0   
2               200507                  0            0           0   
3               200507                  1            1           1   
4               200507                  1            1           1   
...                ...                ...          ...         ...   
15002           201603                 53            0           0   
15003           201603                 53            1           5   
15004           201603                 53            1           5   
15005           201603                 39            1           3   
15006           201603                 39            1           3   

       Price Category Code  Terminal Boarding Area  Passenger Count  Year  \
0                        0         0             B            27271  2005   
1    

In [157]:
#hago lo mismo que con la columna "GEO Summary"
valores_unicos = df["Boarding Area"].unique().compute()

valores_a_reemplazar = ["A", "B", "C", "D", "E", "F", "G", "Other"]
valores_nuevos = np.arange(len(valores_a_reemplazar), dtype=np.int64)

mapping_dict_Boarding_Area = dict(zip(valores_a_reemplazar, valores_nuevos))

df["Boarding Area"] = df["Boarding Area"].map(mapping_dict_Boarding_Area)

df["Boarding Area"] = df["Boarding Area"].astype(np.int64)

resultado = df.compute()
print(resultado)

       Activity Period  Operating Airline  GEO Summary  GEO Region  \
0               200507                  0            0           0   
1               200507                  0            0           0   
2               200507                  0            0           0   
3               200507                  1            1           1   
4               200507                  1            1           1   
...                ...                ...          ...         ...   
15002           201603                 53            0           0   
15003           201603                 53            1           5   
15004           201603                 53            1           5   
15005           201603                 39            1           3   
15006           201603                 39            1           3   

       Price Category Code  Terminal  Boarding Area  Passenger Count  Year  \
0                        0         0              1            27271  2005   
1  

In [158]:
#hago lo mismo que con la columna "GEO Summary"

valores_unicos = df["Month"].unique().compute()

valores_a_reemplazar = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
valores_nuevos = np.arange(len(valores_a_reemplazar), dtype=np.int64)

mapping_dict_Month = dict(zip(valores_a_reemplazar, valores_nuevos))

df["Month"] = df["Month"].map(mapping_dict_Month)

df["Month"] = df["Month"].astype(np.int64)

resultado = df.compute()
print(resultado)

       Activity Period  Operating Airline  GEO Summary  GEO Region  \
0               200507                  0            0           0   
1               200507                  0            0           0   
2               200507                  0            0           0   
3               200507                  1            1           1   
4               200507                  1            1           1   
...                ...                ...          ...         ...   
15002           201603                 53            0           0   
15003           201603                 53            1           5   
15004           201603                 53            1           5   
15005           201603                 39            1           3   
15006           201603                 39            1           3   

       Price Category Code  Terminal  Boarding Area  Passenger Count  Year  \
0                        0         0              1            27271  2005   
1  

In [159]:
#para guardar el diccionario "valor_dict" uso la libreria pickle, y asi poder cargarlo en otro Jupyter notebook
import pickle

with open("DATOS/lista.pkl", "wb") as archivo:
    pickle.dump(valor_dict, archivo)
    pickle.dump(mapping_dict_Boarding_Area, archivo)
    pickle.dump(mapping_dict_Geo_Region, archivo)
    pickle.dump(mapping_dict_Geo_Summary, archivo)
    pickle.dump(mapping_dict_Month, archivo)
    pickle.dump(mapping_dict_Price_Category_Code, archivo)
    pickle.dump(mapping_dict_Terminal, archivo)


In [160]:
#comprobamos que todos los tipos sean de timpo int
df.dtypes

Activity Period        int64
Operating Airline      int64
GEO Summary            int64
GEO Region             int64
Price Category Code    int64
Terminal               int64
Boarding Area          int64
Passenger Count        int64
Year                   int64
Month                  int64
dtype: object

In [161]:
# compruebo si hay valores Nan en cada columna
num_nulos_por_columna = df.isna().sum()

# imprimo la suma de los valores Nan
resultado = num_nulos_por_columna.compute()
print("Número de valores NaN por columna:")
print(resultado)

Número de valores NaN por columna:
Activity Period        0
Operating Airline      0
GEO Summary            0
GEO Region             0
Price Category Code    0
Terminal               0
Boarding Area          0
Passenger Count        0
Year                   0
Month                  0
dtype: int64


In [162]:
#Como no hay valores nulos el dataset estaria ya preparado para analizarlo
#hago una copia del dataset limpio
df.to_csv("DATOS/air_traffic_data_limpio.csv", index=False, single_file=True)

['c:\\Users\\smite\\Documents\\GitHub\\Trabajo-Final-Programacion-Paralela\\DATOS\\air_traffic_data_limpio.csv']